## 시간 흐름에 따른 개발 언어 인기도 분석 

- 오늘날 여전히 사용되는 가장 오랜된 개발 언어는 1957년에 개발된 포트란(FORTRAN)이다. 이후로 많은 개발 언어가 발전해왔다. 그런데 가장 인기 있는 개발 언어는 무엇일까? 
- 이 뜨거운 질문에 대한 대한 답은 스택 오버플로우(Stack Overflow)에서 도움을 받을 수 있다. 스택 오버플로우의 각 게시물은 태그가 함께 제공되며, 개발 언어 이름이 태그가 될 수 있다. 
- 어떤 언어가 가장 인기 있는지 알아내려면 스택 오버플로우에서 각 언어로 태그가 지정된 게시물 수를 계산하면 된다.(가장 많은 게시물을 가진 언어가 바로 인기가 많은 개발 언어라고 할 수 있다.)

### 여기서 배울 내용 
1. 맷플롯립(Matplotlib)으로 데이터를 시각화 하고 차트를 만드는 법
2. 판다스로 데이터를 피벗, 그룹화 및 조작하여 원하는 형식으로 가져오는 방법 
3. 타임스템프 및 시계열 데이터로 작업하는 방법
4. 원하는 대로 꺾은선형 차트의 스타일을 사용자 지정하는 방법

### 데이터를 얻는 법
- 제공된 .csv 파일을 사용하거나 (선택적으로) StackExchange에서 SQL 쿼리를 실행하여 새로운 (가장 신선한?) 데이터를 얻어보아라.
- 제공되는 해당 링크 따라 [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com)에서 쿼리를 실행하여 .csv 파일을 얻어보아라/
- SQL 쿼리문
<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

- 이미 제공된 CSV 파일을 사용해도 무관함

### 1. 데이터 열기 및 탐색

In [1]:
# 필요한 패키지 부르기 
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
# 데이터 판다스로 읽고, 컬럼명 지정하기 
df = pd.read_csv('QueryResults.csv')
df.columns = ['DATE', 'TAG', 'POSTS']
df

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,javascript,162
3,2008-08-01 00:00:00,c,85
4,2008-08-01 00:00:00,python,124
...,...,...,...
1986,2020-07-01 00:00:00,r,5694
1987,2020-07-01 00:00:00,go,743
1988,2020-07-01 00:00:00,ruby,775
1989,2020-07-01 00:00:00,perl,182


In [2]:
# 데이터 불러오기 또다른 방법 - 여기서 헤더 행을 0으로 설정하면 열 자체의 이름으로 대처할 수 있음
df = pd.read_csv('QueryResults.csv', names=['DATE', 'TAG', 'POSTS'], header=0)
df

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,javascript,162
3,2008-08-01 00:00:00,c,85
4,2008-08-01 00:00:00,python,124
...,...,...,...
1986,2020-07-01 00:00:00,r,5694
1987,2020-07-01 00:00:00,go,743
1988,2020-07-01 00:00:00,ruby,775
1989,2020-07-01 00:00:00,perl,182


In [3]:
# 현제 데이터의 행과 열의 개수 파악
df.shape

(1991, 3)

- 해당 csv 파일의 데이터의 총개수는 아래와 같다.
    - 행: 1,991 건
    - 열: 3건

In [4]:
# 각 열에 있는 항목의 수 세기 
df.count()

DATE     1991
TAG      1991
POSTS    1991
dtype: int64

- 만약 열에 있는 항목의 수를 세기 위해서 `.count()`를 사용할 수 있다. count()는 실제로 각 열에 있는 NaN이 아닌 값의 개수를 알려준다.

### 2. 개발 언어 분석 
- 해당 데이터 중 "TAG"는 언어의 이름이다. 예를 들어, 첫 행을 보면 2008년 7월에 C# 언어 태그가 지정된 게시물이 3개가 있었다. "TAG"가 카테고리 역할을 한다고 했을 때 언어당 게시물 수를 계산하는 방법을 알 수는 없을까? 
- 스택 오버플로우가 만들어진 이후, 가장 많은 수의 게시물을 보유한 언어는 무엇일까? 
- 또한 일부 언어는 C만큼 오래되었고, Swift 같은 새로운 언어도 존재한다. 2008년 7월에 시작하므로 일분 언어는 매워 게시물이 전혀 없을 것인데, 각 개발 언어당 게시물이 몇 개월간 존재하는지 세볼 수 있을까?

In [5]:
# 스택 오버플로우에서 가장 많은 게시물을 보유한 언어 탐색
df.groupby('TAG').sum()

,POSTS
TAG,
assembly,34852
c,336042
c#,1423530
c++,684210
delphi,46212
go,47499
java,1696403
javascript,2056510
perl,65286


In [6]:
# 스택 오버플로우에서 개발 언어당 항목이 몇 개월 존재했는지 탐색 
df.groupby('TAG').count()

,DATE,POSTS
TAG,,
assembly,144,144
c,144,144
c#,145,145
c++,144,144
delphi,144,144
go,129,129
java,144,144
javascript,144,144
perl,144,144


- 데이터 탐색 결과는 아래와 같다. 
    - 스택 오버플로우에서 가장 많은 게시물을 보유한 언어는 2,056,510건이 포스트 된 "javascript" 이다. 
    - 각 개발 언어당 게시물이 몇 개월간 존재하는지 살보았다. 그중에서 가장 많은 개월 수를 가진 언어는 145개월인 "C#"이었다.

### 3. 타임스탬프 작업 

#### 3-1. 개별 셀 선택 

In [9]:
# 날짜 두 번째 항목 조회
df['DATE'][1]

'2008-08-01 00:00:00'

- 데이터 프레임 'DATE'열에서 "[]" 표기를 사용해서 열의 n번쨰 항목을 살펴 볼 수 있다. 

In [10]:
# 열의 이름이 공백이 없는 경우 
df.DATE[1]

'2008-08-01 00:00:00'

- 만약 열의 이름에 공백이 없는 경우 "."표기를 사용할 수 있다.

#### 3-2. 데이터 유형 검사

In [11]:
# 'DATE'열 데이터 유형 검사 
type(df['DATE'][1])

str

- 위의 셀 내용을 입력할 때 날짜 개체가 아니라 문자열로 하고 있다는 것을 확인할 수 있다. 문자열 형식이 항상 필요하지 않은 00:00:00을 보여주고 있을 뿐만 아니라 날짜와 시간을 처리해주는 Datetme 개체로 작업하는 이점을 살리지 못하고 있다.

In [12]:
# 문자열에서 Datetime으로 데이터타입 바꾸기 
print(pd.to_datetime(df.DATE[1]))
type(pd.to_datetime(df.DATE[1]))

2008-08-01 00:00:00


pandas._libs.tslibs.timestamps.Timestamp

- 판다스는 "to_datetime()" 메소드를 사용하여 문자열 타임스탬프로 변환하는데 도움을 준다.

In [13]:
# 판다스에서 "to_datetime()"을 사용하여 전체 "DATE"열 반환 
df.DATE = pd.to_datetime(df.DATE)
df.head()

,DATE,TAG,POSTS
0,2008-07-01,c#,3
1,2008-08-01,assembly,8
2,2008-08-01,javascript,162
3,2008-08-01,c,85
4,2008-08-01,python,124
